<a href="https://colab.research.google.com/github/faldongg/PMO-Auto-Fill/blob/main/PMO_AutoFIll.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Liibrary


In [ ]:
import pandas as pd
import os
import re
from google.colab import files
from thefuzz import process, fuzz

#  Fungsi Preprocessing

In [ ]:
def preprocess_text(text):
    """Preprocessing teks untuk matching"""
    if pd.isna(text):
        return ""
    text = str(text).lower().strip()
    if '>' in text:
        parts = text.split('>')
        last = parts[-1].strip()
        if len(last) < 3 and len(parts) > 1:
            text = parts[-2].strip() + " " + last
        else:
            text = last
    return text

#  Load Target File

In [ ]:
# --- 1. LOAD TARGET (HASIL SEBELUMNYA) ---
input_file = 'Hasil_Mapping_FUZZY.xlsx'
output_filename = 'Hasil_Mapping_FINAL.xlsx'

# Cek file input
if not os.path.exists(input_file):
    print(f"⚠️ File '{input_file}' tidak ditemukan.")
    print("Mencoba pakai file asli '2. ItemTambahanKontrakPayung...'")
    input_file = '2. ItemTambahanKontrakPayung 2019-2022 Hasil Mapping.xlsx'

if not os.path.exists(input_file):
    print("❌ File input tidak ada. Silakan upload file hasil terakhir Anda.")
    uploaded = files.upload()
    if uploaded:
        input_file = list(uploaded.keys())[0]
    else:
        raise Exception("File input tidak ditemukan!")

print(f"\n📥 Membaca Target dari: {input_file}")
xls_target = pd.ExcelFile(input_file, engine='openpyxl')

⚠️ File 'Hasil_Mapping_FUZZY.xlsx' tidak ditemukan.
Mencoba pakai file asli '2. ItemTambahanKontrakPayung...'

📥 Membaca Target dari: 2. ItemTambahanKontrakPayung 2019-2022 Hasil Mapping.xlsx


# Pilih Sheet Target

In [ ]:
# --- LOGIKA PENENTUAN SHEET TARGET ---
target_sheet_name = 'ItemTambahanKontarPayung'

if target_sheet_name in xls_target.sheet_names:
    print(f"✅ Sheet '{target_sheet_name}' ditemukan.")
    df_target = pd.read_excel(xls_target, sheet_name=target_sheet_name)
else:
    # Fallback: Ambil sheet pertama
    first_sheet = xls_target.sheet_names[0]
    print(f"⚠️ Sheet '{target_sheet_name}' tidak ada. Menggunakan sheet pertama: '{first_sheet}'")
    df_target = pd.read_excel(xls_target, sheet_name=first_sheet)

print(f"Total baris di Target: {len(df_target)}")

✅ Sheet 'ItemTambahanKontarPayung' ditemukan.
Total baris di Target: 57412


# Load Referensi File

In [ ]:
# --- 2. LOAD REFERENSI (WAJIB DARI FILE ASLI) ---
original_filename = '2. ItemTambahanKontrakPayung 2019-2022 Hasil Mapping.xlsx'

print("\n🔍 Mencari data Referensi...")
if os.path.exists(original_filename):
    xls_ref = pd.ExcelFile(original_filename, engine='openpyxl')
    if 'ItemKontrakPayung' in xls_ref.sheet_names:
        df_ref = pd.read_excel(xls_ref, sheet_name='ItemKontrakPayung')
        print("✅ Referensi dimuat dari file asli lokal.")
    else:
        print("❌ File asli ada, tapi sheet 'ItemKontrakPayung' tidak ditemukan.")
        raise Exception("Sheet referensi tidak ditemukan!")
else:
    print("⚠️ File asli tidak ditemukan (mungkin ini sesi baru).")
    print("   Mohon upload file Excel ASLI (yang ada sheet ItemKontrakPayung)-nya.")
    uploaded = files.upload()
    if uploaded:
        fn = list(uploaded.keys())[0]
        df_ref = pd.read_excel(fn, sheet_name='ItemKontrakPayung')
    else:
        raise Exception("File referensi tidak ditemukan!")

print(f"📊 Status Data: Target={len(df_target)} baris, Referensi={len(df_ref)} baris")


🔍 Mencari data Referensi...
✅ Referensi dimuat dari file asli lokal.
📊 Status Data: Target=57412 baris, Referensi=3269 baris


# Persiapan Mapping

In [ ]:
# --- 3. PERSIAPAN MAPPING ---
df_ref['clean_key'] = df_ref['ItemPekerjaan'].apply(preprocess_text)
df_ref_unique = df_ref.drop_duplicates(subset=['clean_key'])
ref_choices = df_ref_unique['clean_key'].tolist()
ref_map = df_ref_unique.set_index('clean_key')['FORM_ID_KONTRAK_PAYUNG'].to_dict()

print(f"✅ Total referensi unik: {len(ref_choices)}")

✅ Total referensi unik: 2239


# Identifikasi Baris yang Perlu Diisi

In [ ]:
# --- 4. EKSEKUSI MATCHING ---
col_target_fill = 'FORM_ID (KontrakPayung)'
col_key = 'ItemPekerjaan'

# Deteksi baris yang perlu diisi
def is_target(val):
    s = str(val).lower().strip()
    return s in ['skip', 'nan', '', 'tidak_ditemukan', 'data_tidak_valid'] or pd.isna(val)

mask_to_fill = df_target[col_target_fill].apply(is_target)
indices = df_target[mask_to_fill].index

total_items = len(indices)
print(f"\n🎯 Target sisa yang harus diisi: {total_items} baris.")


🎯 Target sisa yang harus diisi: 11080 baris.


# Eksekusi Fuzzy Matching

In [ ]:
matches_found = 0
memo = {}

print("⚙️ Memulai FUZZY MATCHING FINAL...")

for i, idx in enumerate(indices):
    original_text = df_target.at[idx, col_key]
    query_text = preprocess_text(original_text)

    if not query_text or len(query_text) < 2:
        df_target.at[idx, col_target_fill] = "DATA_TIDAK_VALID"
        continue

    if query_text in memo:
        best_match, score = memo[query_text]
    else:
        result = process.extractOne(query_text, ref_choices, scorer=fuzz.token_set_ratio)
        if result:
            best_match, score = result
            memo[query_text] = (best_match, score)
        else:
            best_match, score = None, 0

    # Threshold: 60
    if score >= 60:
        val = ref_map.get(best_match)
        if val:
            df_target.at[idx, col_target_fill] = val
            matches_found += 1
        else:
            df_target.at[idx, col_target_fill] = "None"
    else:
        df_target.at[idx, col_target_fill] = "None"

    if (i + 1) % 500 == 0:
        print(f"   ...Sisa diproses {i + 1}/{total_items} | Match Baru: {matches_found}")

print(f"\n✅ SELESAI!")
print(f"   Total tambahan match: {matches_found}")

⚙️ Memulai FUZZY MATCHING FINAL (Threshold 75)...
   ...Sisa diproses 500/11080 | Match Baru: 29
   ...Sisa diproses 1000/11080 | Match Baru: 29
   ...Sisa diproses 1500/11080 | Match Baru: 29
   ...Sisa diproses 2000/11080 | Match Baru: 29
   ...Sisa diproses 2500/11080 | Match Baru: 29
   ...Sisa diproses 3000/11080 | Match Baru: 29
   ...Sisa diproses 3500/11080 | Match Baru: 29
   ...Sisa diproses 4000/11080 | Match Baru: 39
   ...Sisa diproses 4500/11080 | Match Baru: 109
   ...Sisa diproses 5000/11080 | Match Baru: 145
   ...Sisa diproses 5500/11080 | Match Baru: 150
   ...Sisa diproses 6000/11080 | Match Baru: 204
   ...Sisa diproses 6500/11080 | Match Baru: 214
   ...Sisa diproses 7000/11080 | Match Baru: 314
   ...Sisa diproses 7500/11080 | Match Baru: 378
   ...Sisa diproses 8000/11080 | Match Baru: 393
   ...Sisa diproses 8500/11080 | Match Baru: 433
   ...Sisa diproses 9000/11080 | Match Baru: 471
   ...Sisa diproses 9500/11080 | Match Baru: 528
   ...Sisa diproses 10000/11

# Save & Download

In [ ]:
print(f"💾 Menyimpan ke '{output_filename}'...")
df_target.to_excel(output_filename, index=False)
files.download(output_filename)
print("🎉 File berhasil diunduh!")

💾 Menyimpan ke 'Hasil_Mapping_FINAL.xlsx'...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🎉 File berhasil diunduh!
